## Syntax-dependent trace



In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
CACHE_DIR = "/scratch/lucchetti.f/models/"
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR


import sys
sys.path.append('..')

from model_utils import check_devs, clear_devs
import tqdm as notebook_tqdm
from model import ModelLoader
# from model_loader import ModelLoader
# from model_manager import ModelManager

import gc
import torch


/home/lucchetti.f/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
check_devs()

0 / 34079899648 used for device 0, reserved 0


In [7]:
# MODEL_NAME = "Salesforce/codegen-16B-mono"
MODEL_NAME = "bigcode/santacoder"

## Optional Download the Weights
# weights_location = hf_hub_download(MODEL_NAME, "pytorch_model.bin", cache_dir=CACHE_DIR)
# hf_hub_download(MODEL_NAME, "config.json", cache_dir=CACHE_DIR)

In [11]:
# loaded_model = ModelLoader(MODEL_NAME)
mt = ModelLoader(MODEL_NAME)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


AttributeError: 'GPT2CustomConfig' object has no attribute 'n_ctx'

In [6]:
# mt.model

In [7]:
check_devs()

32206917632 / 34089926656 used for device 0, reserved 32212254720


## Inference

In [8]:
from model_utils import generate_fast

model = mt.model
tokenizer = mt.tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [9]:
prompts = ["""# intialize three integers
           int x = 1
           int y = 2
           int z ="""]

prompts = ["""int x ="""]

# print(inputs)
# inputs = inputs.to('cpu')
# print(inputs)
# print(inputs.shape)
tokenized = tokenizer(prompts, return_tensors="pt")

outputs = model.generate(tokenized.input_ids.to(model.device), max_new_tokens=2, top_k = 1)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


int x = 0;


/home/lucchetti.f/.local/lib/python3.7/site-packages/transformers/models/codegen/modeling_codegen.py:167: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541035/work/aten/src/ATen/native/TensorCompare.cpp:413.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


## Trace + activation patch

In [13]:
txt, ret_dict = mt.generate(
    prompts,
    argmax_greedy= True,
    debug = True,
    request_activations= [mt.layer_name_format.format(i) for i in range(5,10)]
)
txt

prompt <0> ==> [(' 0', 0.2045), (' (', 0.0966), (' 1', 0.0852), (' 2', 0.0334), (' {', 0.0314)]


RuntimeError: The size of tensor a (6) must match the size of tensor b (3) at non-singleton dimension 3